In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

#BiocManager::install("treeio")
#BiocManager::install("ggtree")
#BiocManager::install("Biostrings")
#BiocManager::install("XVector")

#install.packages("tidyverse")
#install.packages("ggplot2")
#install.packages("ggstance")
#install.packages("ggmsa")
#install.packages("seqmagick")
#install.packages("shiny")

In [1]:
library(treeio)
library(ggtree)
library(tidyverse)
library(ggplot2)
library(ggstance)
library(Biostrings)
library(ggmsa)
library(seqmagick)
library(XVector)
library(gtable)
library(grid)
library(shiny)

Registered S3 method overwritten by 'treeio':
  method     from
  root.phylo ape 

treeio v1.13.1  For help: https://yulab-smu.github.io/treedata-book/

If you use treeio in published research, please cite:

LG Wang, TTY Lam, S Xu, Z Dai, L Zhou, T Feng, P Guo, CW Dunn, BR Jones, T Bradley, H Zhu, Y Guan, Y Jiang, G Yu. treeio: an R package for phylogenetic tree input and output with richly annotated and associated data. Molecular Biology and Evolution 2020, 37(2):599-603. doi: 10.1093/molbev/msz240


ggtree v2.3.7  For help: https://yulab-smu.github.io/treedata-book/

If you use ggtree in published research, please cite the most appropriate paper(s):

- Guangchuang Yu. Using ggtree to visualize data on tree-like structures. Current Protocols in Bioinformatics, 2020, 69:e96. doi:10.1002/cpbi.96
- Guangchuang Yu, Tommy Tsan-Yuk Lam, Huachen Zhu, Yi Guan. Two methods for mapping and visualizing associated data on phylogeny using ggtree. Molecular Biology and Evolution 2018, 35(12):3041-3

## Likely, the simplest shiny app ever

In [2]:
server <- function(input, output, session) { } #the server

ui <- basicPage("This is a real Shiny app and jupyter is the best thing ever ;D") # the user interface

shinyApp(ui = ui, server = server) # this launches your app


Listening on http://127.0.0.1:7254



## Ok so let's build a web app which displays some trees
## overview: 4 parts - preprocessing, ui, server, app calling

### preporcessing

In [4]:
## define my two classes - defining the A2TEA HYPOTHESES object ;D
# class for the expanded_OG - containing all different types of data we have on it
setClass("expanded_OG", slots=list(genes="spec_tbl_df", 
                                  fasta_files="list", 
                                  msa="AAMultipleAlignment", 
                                  tree="phylo"))

# class for the hypotheses
setClass("hypothesis", slots=list(description="character", 
                                  number="character",
                                  expanded_in ="character", 
                                  compared_to="character", 
                                  expanded_OGs="list"))


#load the A2TEA input
#load("/home/tyll/Desktop/PhD/A2TEA/shiny/example_trees/A2TEA_finished.RData")

### ui

In [265]:
# Define UI for tree app ----

#hypothesesList <- as.list(hypotheses$hypothesis)
#names(hypothesesList) <- hypotheses$name
#hypothesesList


ui <- pageWithSidebar(
  
  # App title ----
  headerPanel("Tree building"),
  
  # Sidebar panel for inputs ----
  sidebarPanel(
  
      fileInput(inputId = "hypotheses",
                label = "Upload hypotheses.tsv file:",
                multiple = FALSE,
                accept = ".tsv"), 
      
      fileInput(inputId = "HYPOTHESES",
                label = "Upload HYPOTHESES.a2tea.rds file:",
                multiple = FALSE,
                accept = ".rds"), 
      
      fileInput(inputId = "HOG_DE",
                label = "Upload HOG_DE.a2tea.rds file:",
                multiple = FALSE,
                accept = ".rds"), 
      
        # Input: Selector for variable to plot against mpg ----
      selectInput("select",
                    label = h3("Select hypothesis:"),
                    "") 
#                    choices = c("A", "B", "C"))
#                    choices = textOutput("hypothesesList"))
#                    choices = hypothesesList)
  ),
  
  # Main panel for displaying outputs ----
  mainPanel(
      h3(textOutput("text"))
  )
)

In [71]:
#?selectInput
#create input
#mylist <- as.list(paste0("hypothesis_", hypotheses$hypothesis))
#names(mylist) <- hypotheses$name
#mylist

#accessing the hypotheses
#HYPOTHESES.a2tea$hypothesis_1@expanded_in
#mylist[[1]]
#HYPOTHESES.a2tea[[mylist[[1]]]]@expanded_in

select
#HYPOTHESES.a2tea[[eval(parse(select$input))]]@compared_to

function (.data, ...) 
{
    UseMethod("select")
}
<bytecode: 0x5563ba447f38>
<environment: namespace:dplyr>

### server

In [283]:
#increasing maximum file size for upload
#https://stackoverflow.com/questions/18037737/how-to-change-maximum-upload-size-exceeded-restriction-in-shiny-and-save-user
options(shiny.maxRequestSize=50*1024^2)

myDataFrame <- data.frame(names=c(""))


server <- function(input, output, session) {
    
    tsv <- reactive({
     inFile <- input$hypotheses
     if (is.null(inFile)) {
         d <- myDataFrame
     } else {
         d <- read_tsv(input$hypotheses$datapath)
        }
        d
    })  
    
 
    output$hypothesesList <- renderText({
        tsv()$name
    })
    
    observe({
       updateSelectInput(session, "select",
                         label = "Select Hypothesis:",
                         choices = tsv()$name,
                         selected = tsv()$name[1])

    
    
    
    
    # read-in the .RData object
    data <- reactive({
       readRDS(input$HYPOTHESES$datapath)
    })
    
    hypothesisNUM <- reactive({
#        data()[[as.numeric(input$select)]]@number
        data()[[hypotheses$hypothesis[hypotheses$name==input$select]]]@number
    })
   
      output$text <- renderText({
        hypothesisNUM()
    })
    
#    output$recFacetPlot <- renderPlot({
        
#  })

    })
    
}

In [232]:
#saveRDS(HYPOTHESES.a2tea, "example_trees/HYPOTHESES.a2tea.rds")
#HYPOTHESES.a2tea[[testSelect]]@compared_to
read_tsv("example_trees/hypotheses.tsv")$name


── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────
cols(
  hypothesis = col_double(),
  name = col_character(),
  expanded_in = col_character(),
  compared_to = col_character()
)




[1] "Expanded in Arabidopsis compared to Monocots"
[2] "Expanded in barley compared to maize"

### app calling

In [284]:
shinyApp(ui, server)


Listening on http://127.0.0.1:5402

Warning message:
“Error in readRDS: bad 'file' argument”

── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────
cols(
  hypothesis = col_double(),
  name = col_character(),
  expanded_in = col_character(),
  compared_to = col_character()
)


Warning message:
“Error in readRDS: bad 'file' argument”


### playground & working examples

In [282]:
hypotheses$hypothesis[hypotheses$name=="Expanded in barley compared to maize"]

[1] 2

In [247]:
saveRDS( data.frame(names=c("Jill","Jane","Megan")),"example_trees/myDataFrame.rds")

#### completely working mini application showcasing runtime loading of data and reactive dropdown

In [249]:
myDataFrame <- data.frame(names=c("Tom","Dick","Harry"))

ui <- shinyUI(
    fluidPage(
        fileInput("file1", "Choose file to upload", accept = ".rds"),
        selectInput("myNames","Names", ""),
        tableOutput("contents")
    )
)

server <- function(input, output, session) {

    myData <- reactive({
        inFile <- input$file1
        if (is.null(inFile)) {
            d <- myDataFrame
        } else {
            d <- readRDS(inFile$datapath)
        }
        d
    })

    output$contents <- renderTable({
        myData()
    })

    observe({
         updateSelectInput(session, "myNames",
                           label = "myNames",
                           choices = myData()$names,
                           selected = myData()$names[1])
    })

}

shinyApp(ui, server)



Listening on http://127.0.0.1:5402

